In [1]:
from QuantLib import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
depo_maturities = [Period(6,Months), Period(12, Months)]
depo_rates = [0.0, 0.005]
# Bond rates
bond_maturities = [Period(6*i, Months) for i in range(3,21)]
bond_rates = [0.007, 0.009, 0.012, 0.015, 0.019, 0.024, 0.025, 0.027, 0.028,
0.029, 0.03, 0.035, 0.04, 0.05, 0.06, 0.07, 0.075, 0.08]


In [3]:
depo_maturities

[Period("6M"), Period("1Y")]

In [4]:
calc_date = Date(15, 1, 2018)
Settings.instance().evaluationDate = calc_date
calendar = UnitedStates()
business_convention = Unadjusted
day_count = Thirty360()
end_of_month = True
settlement_days = 0
face_amount = 100
coupon_frequency = Period(Semiannual)
settlement_days = 0

In [5]:
depo_helpers = [DepositRateHelper(QuoteHandle(SimpleQuote(r/100.0)),
                m,
                settlement_days,
                calendar,
                business_convention,
                end_of_month,
                day_count )
for r, m in zip(depo_rates, depo_maturities)]

In [6]:
bond_helpers = []
for r, m in zip(bond_rates, bond_maturities):
    termination_date = calc_date + m
    schedule = Schedule(calc_date,
                   termination_date,
                   coupon_frequency,
                   calendar,
                   business_convention,
                   business_convention,
                   DateGeneration.Backward,
                   end_of_month)

    helper = FixedRateBondHelper(QuoteHandle(SimpleQuote(face_amount)),
                                        settlement_days,
                                        face_amount,
                                        schedule,
                                        [r/100.0],
                                        day_count,
                                        business_convention,
                                        )
    bond_helpers.append(helper)

In [7]:
rate_helpers = depo_helpers + bond_helpers
yieldcurve = PiecewiseLogCubicDiscount(calc_date, rate_helpers, day_count)

In [8]:
# get spot rates
spots = []
tenors = []
for d in yieldcurve.dates():
    yrs = day_count.yearFraction(calc_date, d)
    compounding = Compounded
    freq = Semiannual
    zero_rate = yieldcurve.zeroRate(yrs, compounding, freq)
    tenors.append(yrs)
    eq_rate = zero_rate.equivalentRate(day_count,
                                       compounding,
                                       freq,
                                       calc_date,
                                       d).rate()
    spots.append(100*eq_rate)

In [9]:
def print_curve(xlist, ylist, precision=3):
    """
    Method to print curve in a nice format
    """
    print("----------------------")
    print("Maturities\tCurve")
    print("----------------------")
    for x,y in zip(xlist, ylist):
        print(x,"\t\t", round(y, precision))
    print("----------------------")

In [ ]:
print(tenors)

In [10]:
print_curve(tenors,spots)

----------------------
Maturities	Curve
----------------------
0.0 		 0.0
0.5027777777777778 		 0.0
1.0027777777777778 		 0.005
1.5 		 0.007
2.0 		 0.009
2.5 		 0.012
3.0 		 0.015
3.5 		 0.019
4.0 		 0.024
4.5 		 0.025
5.0 		 0.027
5.5 		 0.028
6.0 		 0.029
6.5 		 0.03
7.0 		 0.035
7.5 		 0.04
8.0 		 0.05
8.5 		 0.06
9.0 		 0.07
9.5 		 0.075
10.0 		 0.08
----------------------


In [ ]:
yieldcurve.dates()

In [ ]:
print(spots)

# Bond Valuation

In [11]:
spot_dates = yieldcurve.dates()
spot_rates = spots
interpolation = Linear()
compounding = Compounded
compounding_frequency = Annual
spot_curve = ZeroCurve(spot_dates, spot_rates, day_count, calendar,
interpolation, compounding, compounding_frequency)
spot_curve_handle = YieldTermStructureHandle(spot_curve)

In [17]:
issue_date = Date(15, 1, 2018)
maturity_date = Date(15, 1, 2020)
tenor = Period(Semiannual)
date_generation = DateGeneration.Backward
month_end = False
schedule = Schedule (issue_date, maturity_date, tenor,
calendar, business_convention,
business_convention , date_generation,
month_end)

In [18]:
list(schedule)

[Date(15,1,2018),
 Date(15,7,2018),
 Date(15,1,2019),
 Date(15,7,2019),
 Date(15,1,2020)]

In [19]:
coupon_rate = .06
coupons = [coupon_rate]
settlement_days = 0
face_value = 100
fixed_rate_bond = FixedRateBond(settlement_days,face_value,schedule,coupons,day_count)

In [20]:
bond_engine = DiscountingBondEngine(spot_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

In [21]:
fixed_rate_bond.NPV()

110.12721566547108